In [6]:
%matplotlib inline

import os
import re
import sys
import h5py
import numpy
import subprocess
try:
    import cPickle as pickle
except: 
    import pickle

import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

from mpl_toolkits.basemap import Basemap

from copy import deepcopy
from oqmbt.oqt_project import OQtProject, OQtModel, OQtSource

from openquake.hmtk.seismicity.occurrence.weichert import Weichert
from openquake.hmtk.seismicity.completeness.comp_stepp_1971 import Stepp1971 
from openquake.hmtk.plotting.seismicity.completeness.plot_stepp_1972 import create_stepp_plot
from openquake.hmtk.plotting.seismicity.occurrence.recurrence_plot import plot_trunc_gr_model
from openquake.hmtk.plotting.seismicity.catalogue_plots import (plot_magnitude_time_scatter, 
                                                      plot_magnitude_time_density)
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueParser
from openquake.hmtk.seismicity.selector import CatalogueSelector
from openquake.hmtk.seismicity.declusterer.dec_gardner_knopoff import GardnerKnopoffType1
from openquake.hmtk.seismicity.declusterer.distance_time_windows import GardnerKnopoffWindow

In [7]:
base = os.getcwd()
prj_path_long = os.path.join("/Users/kjohnson/GEM/Regions/paisl18/project/paisl.oqmbtp")
prj_path = os.path.normpath(prj_path_long)
os.environ["OQMBT_PROJECT"] = prj_path


In [8]:
#
# load project information 
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
oqtkp.directory = os.path.dirname(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
catalogue_csv_filename = os.path.join(oqtkp.directory, getattr(model,'catalogue_csv_filename'))
#
# info
text_length = 30
print ('{:s}: {:s}'.format('Model id'.ljust(text_length), model_id))
print ('{:s}: {:s}'.format('Reading'.ljust(text_length), catalogue_csv_filename))

Model id                      : model01
Reading                       : /Users/kjohnson/GEM/Regions/paisl18/project/../data/catalogues/locations/FinalCatalog/PI_final.hmtk


In [9]:
# cutoff magnitude is the lowest magnitude treshold. All the events below this threshold 
# are filtered out at the beginning of the processing workflow
cutoff_magnitude = float(model.catalogue_cutoff_magnitude)
#
# width of bins used to discretise the MFDs
binwidth = float(model.mfd_binwidth)
#
# printing info
print ('{:s}: {:.2f}'.format('MFD bin width'.ljust(text_length), binwidth))
print ('{:s}: {:.2f}'.format('Magnitude cutoff'.ljust(text_length), cutoff_magnitude))

MFD bin width                 : 0.10
Magnitude cutoff              : 3.50


In [10]:
#
# parsing the .csv catalogue
catalogue_csv_filename = '/Users/kjohnson/GEM/Regions/paisl18/data/catalogues/locations/FinalCatalog/PI_final.hmtk'
catalogue_parser = CsvCatalogueParser(catalogue_csv_filename)
catalogue = catalogue_parser.read_file()
selector = CatalogueSelector(catalogue, create_copy=False)
selector.within_magnitude_range(cutoff_magnitude, 10.)
dep_max = float(model.catalogue_maximum_depth)
dep_min = float(model.catalogue_minimum_depth)
tmp = selector.within_depth_range(dep_max, dep_min)
#
# info
print ('Catalogue:')
print ('    minimum hypocentral depth: {:.2f}'.format(numpy.min(catalogue.data['depth'])))
print ('    maximum hypocentral depth: {:.2f}'.format(numpy.max(catalogue.data['depth'])))

Catalogue Attribute locId is not a recognised catalogue key
Catalogue Attribute field_27 is not a recognised catalogue key
Catalogue Attribute field_28 is not a recognised catalogue key
Catalogue Attribute 360_1 is not a recognised catalogue key
Catalogue Attribute field_26 is not a recognised catalogue key
Catalogue Attribute field_24 is not a recognised catalogue key
Catalogue Attribute MagId is not a recognised catalogue key
Catalogue Attribute field_29 is not a recognised catalogue key
Catalogue Attribute field_25 is not a recognised catalogue key
Catalogue Attribute field_23 is not a recognised catalogue key
Catalogue Attribute field_30 is not a recognised catalogue key
Catalogue Attribute Identifier is not a recognised catalogue key
Catalogue:
    minimum hypocentral depth: 0.00
    maximum hypocentral depth: 34.90


/Users/kjohnson/GEM/oq-engine/openquake/hmtk/seismicity/selector.py:380: RuntimeWarning: invalid value encountered in greater_equal
  is_valid = np.logical_and(self.catalogue.data['depth'] >= upper_depth,
/Users/kjohnson/GEM/oq-engine/openquake/hmtk/seismicity/selector.py:381: RuntimeWarning: invalid value encountered in less
  self.catalogue.data['depth'] < lower_depth)


In [19]:
distance_time_wind = GardnerKnopoffWindow()
config = {'time_distance_window': distance_time_wind, 'fs_time_prop': .9}
declusterer = GardnerKnopoffType1()
from openquake.hmtk.seismicity.utils import decimal_year, haversine
import numpy as np

In [32]:
neq = len(catalogue.data['magnitude'])
year_dec = decimal_year(
            catalogue.data['year'], catalogue.data['month'],
            catalogue.data['day'])
sw_space, sw_time = (
            config['time_distance_window'].calc(catalogue.data['magnitude']))
# Initial Position Identifier
eqid = np.arange(0, neq, 1)
# Pre-allocate cluster index vectors
vcl = np.zeros(neq, dtype=int)
# Sort magnitudes into descending order
id0 = np.flipud(np.argsort(catalogue.data['magnitude'],
                           kind='heapsort'))
longitude = catalogue.data['longitude'][id0]
latitude = catalogue.data['latitude'][id0]
sw_space = sw_space[id0]
sw_time = sw_time[id0]
year_dec = year_dec[id0]
eqid = eqid[id0]
flagvector = np.zeros(neq, dtype=int)
# Begin cluster identification
clust_index = 0


7523


In [35]:
for i in range(0, 10):
    if vcl[i] == 0:
        # Find Events inside both fore- and aftershock time windows
        dt = year_dec - year_dec[i]
        print(dt)
        vsel = np.logical_and(
            vcl == 0,
            np.logical_and(
                dt >= (-sw_time[i] * config['fs_time_prop']),
                dt <= sw_time[i]))
        # Of those events inside time window,
        # find those inside distance window
        vsel1 = haversine(longitude[vsel],
                          latitude[vsel],
                          longitude[i],
                          latitude[i]) <= sw_space[i]
        #print(vsel1)
        vsel[vsel] = vsel1[:, 0]
        temp_vsel = np.copy(vsel)
        temp_vsel[i] = False
        if any(temp_vsel):
            # Allocate a cluster number
            vcl[vsel] = clust_index + 1
            flagvector[vsel] = 1
            ## For those events in the cluster before the main event,
            ## flagvector is equal to -1
            temp_vsel[dt >= 0.0] = False
            flagvector[temp_vsel] = -1
            flagvector[i] = 0
            clust_index += 1



[  0.           3.3890411    1.99726027 ...,  91.97808219  94.84109589
  95.72054795]
[ -3.3890411    0.          -1.39178082 ...,  88.5890411   91.45205479
  92.33150685]
[ -1.99726027   1.39178082   0.         ...,  89.98082192  92.84383562
  93.72328767]
[-92.41369863 -89.02465753 -90.41643836 ...,  -0.43561644   2.42739726
   3.30684932]
[-89.91780822 -86.52876712 -87.92054795 ...,   2.06027397   4.92328767
   5.80273973]
[ -0.15342466   3.23561644   1.84383562 ...,  91.82465753  94.68767123
  95.56712329]
[-83.54520548 -80.15616438 -81.54794521 ...,   8.43287671  11.29589041
  12.17534247]
[-89.00547945 -85.61643836 -87.00821918 ...,   2.97260274   5.83561644
   6.71506849]
[-21.99726027 -18.60821918 -20.         ...,  69.98082192  72.84383562
  73.72328767]
[  15.7260274    19.11506849   17.72328767 ...,  107.70410959  110.56712329
  111.44657534]


In [36]:
vcl, flag = declusterer.decluster(catalogue, config)

TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions